[topos]: https://github.com/opeltre/topos
[fp]: https://github.com/opeltre/fp
[torch]: https://pytorch.org

# I. Sheaves and Functors

The [topos] library relies on category theory abstractions
most common in functional languages, 
although quite absent from the usual python paradigms. 

As first motivating example, let us mention the category ${\bf Type}$ 
implemented by a functional language. 
Its _objects_ are compiler-compliant variable types $A$ and its _arrows_ represent typed programs $f : A \to B$ with input in $A$ and 
output in $B$.

The concepts of categories, functors and natural transformations 
originating from algebraic topology, it is only natural to 
follow this ubiquitous line of thought when implementing discrete
topological structures.
However the python language, although great for its ecosystem, does not lend itself easily to type polymorphism.
 
The [topos] package hence relies on the [fp] package, a small stand-alone 
consisting mostly of metaclass definitions and providing typed tensors 
wrapping [torch] instances.
It should have landed on your system when [installing topos][topos]. 

In [1]:
import fp
import topos


## 1. Categories

A category simply consists of a collection of objects and a collection of composable arrows between them. 

>__Definition 1.__
>A _category_ $\bf Cat$ is defined by: 
>- a set or class of _objects_ ${\rm Obj}({\bf Cat})$ 
>- a set of _arrows_ written ${\rm Hom}(A, B)$ or ${\bf Cat}_{A \to B}$ between any two objects >$A$ and $B$,
>- an associative composition defining
>$gf : A \to C$ for all arrows $f : A \to B$ and $g : B \to C$,
>- a unit arrow $1_A : A \to A$ satisfying $1_A 1_A = 1_A$ for every object $A$.
>  


__N.B.__ Associativity demands that $h(gf) = (hg)f$ for any triplet $f, g, h$ of 
composable arrows. 
We may drop brackets when performing multiple compositions 
and simply write $hgf$. 

The canonical example is the category ${\bf Set}$ whose objects are sets and 
arrows $f: A \to B$ are functions from $A$ to $B$. 
Families of mathematical objects 
(e.g. groups, vector spaces, topological spaces, manifolds, ...) 
are also normally defined 
along with the admissible transformations between them, 
i.e. concrete mappings on their underlying sets 
that preserve a relevant structure. 
These categories (${\bf Grp, Vect, Top, Mfd}$, ...) are called _concrete_ 
to mean that they can be naturally embedded in the category of sets by what
we'll call a _forgetful functor_. 


Interestingly, many mathematical objects are also categories themselves, 
because categories are a basic abstraction for transitivity and associativity. 
Let us provide a few more examples of interest below.
The first one plays a major role in the `topos` library, 
as topological structures will be understood as partial orders $K$ embedded in the power set 
${\mathcal P}(\Omega) = 2^\Omega$ of a given set of vertices $\Omega$.

__Example 1.__
A partial order $(K, \leq)$ defines a category ${\bf K}$ in the following way:
- ${\rm Obj}({\bf K}) = K$ 
- ${\bf K}_{a \to b} = \{ \bullet \}$ if $a \leq b$ else $\varnothing$ 

In the notation above, $\{\bullet\}$ defines any _point_ i.e. set of one element.
In other words, we may identify $K$ with the set of objects and relations $a \leq b$ with 
the unique arrow $a \to b$ living in ${\bf K}_{a \to b}$, whenever it is non-empty.
The composition axiom expresses _transitivity_ i.e. 
$a \leq b$ and $b \leq c$ implies $a \leq c$. The unit axiom expresses _reflexivity_ i.e. 
$a \leq a$ for all $a$.

__Example 2.__ A group $G$ defines a category ${\bf G}$ in the following way:
- ${\rm Obj}({\bf G}) = \{ \bullet \}$
- ${\bf G_{\bullet \to \bullet}} = G$
- composition in ${\bf G}_{\bullet \to \bullet}$ is the composition of $G$,
- the unit $1_\bullet$ is the unit of $G$.

Note that all elements of $G$ are composable because they have the same source and target $\bullet$. The structure above is exactly that of a _monoid_, 
i.e. sets equipped with an associative composition: 
one should only demand that every arrow of ${\bf G}_{\bullet \to \bullet}$ 
be inversible to define groups as those special categories above a single object.

The list of examples could be very long...
They'll become more interesting with the notion of functors, 
describing relationships _between_ categories, which we introduce below. 

Let us first quickly show how "arrows" i.e. functions are handled within
[fp](https://github.com/opeltre/fp). Notice how `Arrow(a, b)` provides 
quite more than type decoration, by implementing e.g. composition and [curryfication].

[curryfication]: https://fr.wikipedia.org/wiki/Curryfication

In [2]:
""" Arrows """

from fp import Arrow, Str, Int

#-- Arrow is a binary type constructor --
assert isinstance(Arrow(Int, Int), type)

#-- Arrow creation --

foo = Arrow(Str, Int)(len)

@Arrow(Int, Str)
def bar(x):
    return '|' * x

#-- Arrow composition -- 
assert isinstance(foo @ bar, Arrow(Int, Int))
assert isinstance(bar @ foo, Arrow(Str, Str))

print(bar @ foo @ 'hellow world!')

#-- Arrow curryfication
@Arrow((Int, Int), Int)
def add(x, y): return x + y

assert isinstance(add(2), Arrow(Int, Int))
assert add(2)(6) == add(2, 6)
add(2)


'|||||||||||||'


Int -> Int : add 2

## 2. Functors

Functors map source categories to target categories by acting on both objects and arrows.

>__Definition 2.__
>A functor $F : {\bf C} \to {\bf C'}$ is defined by:
>- an object $F_a$ of ${\bf C'}$ for every object $a$ of ${\bf C}$,
>- an arrow $F_{a \to b} : F_a \to F_b$ of ${\bf C'}$ for every arrow $a \to b$ in ${\bf C}$
>
>such that $F_g F_h = F_{gh}$ for all $h \in {\bf C}_{a \to b}$
>and $g \in {\bf C}_{b \to c}$.

If $(K, \leq)$ and $(K', \leq)$ are two partial orders as in example 1, then 
a functor from ${\bf K}$ to ${\bf K'}$ is an order-preserving 
set map from $K$ to $K'$.
Such maps define the arrows or _morphisms_ of the category of partial orders ${\bf Ord}$.

If $G$ and $G'$ are two groups as in example 2, then 
a functor from ${\bf G}$ to ${\bf G'}$ is a group morphism from $G$ to $G'$. 
Such maps define the arrows of the category of groups ${\bf Grp}$.

Before giving other examples, let us introduce notion of opposite or _dual_ category, 
usefull for describing functors that reverse the direction of arrows.

>__Definition 3.__ 
> To any category ${\bf C}$ we associate the dual category ${\bf C}^{op}$ defined by:
> - ${\rm Obj}({\bf C}^{op}) = {\rm Obj}({\bf C})$
> - ${\bf C}^{op}_{b \to a} \simeq {\bf C}_{a \to b}$ through a 1-1 correspondence written 
> $f^{op} \leftrightarrow f$
> - $f^{op} g^{op} = (gf)^{op}$ for every $f \in {\bf C}_{a \to b}$ and 
$g \in {\bf C}_{a \to b}$,
> - $1_a^{op} = 1_a$ for every object $a$ of ${\bf C}$ and ${\bf C}^{op}$.
>
> A _contravariant_ functor from ${\bf C}$ to ${\bf C'}$ is a functor 
> $F : {\bf C}^{op} \to {\bf C'}$.

A contravariant functor $F : {\bf C}^{op} \to {\bf C'}$ 
simply maps an arrow $a \to b$ of ${\bf C}$ 
to a reversed arrow $F_{b \to a}$ of ${\bf C'}$. 

__Example 4.__
The functor of real valued functions 
$(\:\cdot \to {\mathbb R}) : {\bf Set}^{op} \to {\bf Alg}_c$, contravariant from ${\bf Set}$ to 
the category of commutative $\mathbb R$-algebras ${\bf Alg}_c$, is defined by:
- $\mathbb{R}^A = A \to \mathbb{R}$ is the algebra of functions on $A$,
- $f^* = ((A \overset{f}{\to} B) \to \mathbb{R})$ is the _pullback by_ $f$, also called 
precomposition $f^* : g \mapsto gf$.

Note that one may decline the above in many different flavours, e.g. 
- $C(\:\cdot\:): {\bf Top}^{op} \to {\bf Alg}_c$ restricted to topological spaces and continuous morphisms, 
- $C^\infty(\:\cdot\:) : {\bf Mfd}^{op} \to {\bf Alg}_c$ restricted to smooth manifolds and 
infinitely differentiable maps,
- ...

Example 4 will be of central interest to us, at it is the relevant abstraction for 
assigning a vector space (and algebra) to any finite set, and a linear map (and algebra morphism) to any index mapping. We shall get a reversed linear map by transposition (__not__ an algebra morphism), called _pushforward_, and corresponding to the direct image of measures (integration on fibers). 



Before coming to its implementation, let's briefly describe functors from the programmatic point of view. In python we'll write `F(a)` for an image object and `F.fmap(f)` for an image arrow.
This follows the naming conventions of functional languages, where a functor 
$F : {\bf Type} \to {\bf Type}$ is usually presented as
- a type `F a` for every input type `a`
- a function `F.fmap : (a -> b) -> F a -> F b`

Here is the simplest example of functor encountered in programming:

__Example 3.__
The `List` functor, also written `[.]` , is defined by:
- `List a = [a]` is the type of lists containing elements of type `a`,
- `List.fmap(f)` returns the list of element images by iterating over the input list. 

Here is an example of how this is emulated within python in the
[fp](https://github.com/opeltre/fp) package:

In [3]:
""" List functor """

from fp import List

x = ["tomatoes", "eggplants", "zukinis"]
y = List.fmap(foo)(x)

assert type(List.fmap(foo)) == Arrow(List(Str), List(Int))
print(repr(List.fmap(foo)))
print(repr(y))

List Str -> List Int : map len
List Int : [8, 9, 7]


Let's now come to the tensor types 
returned by the `Tens` functor.
They all subclass a common `Tensor` type wrapping 
`torch.Tensor` instances (have a look at [fp/instances/tens.py] 
for more details on how this is done using a custom `Wrap` monad on algebraic types). 

Note how this is a particular case of example 4., 
the functor 
$(\:\cdot \to \mathbb{R}) : {\bf Set}^{op} \to {\bf Alg}_c$
being restricted to finite domains, viewed as a discrete torus 
$\prod_{i} \mathbb{Z} / n_i \mathbb{Z}$ defined by some shape vector $n$.

[fp/instances/tens.py]: https://github.com/opeltre/fp/instances/tens.py

In [15]:
""" Tens functor """

from fp import Tens
import torch

E = Tens([3, 3])
F = Tens([6, 6])

assert isinstance(E, type)

x = E.ones()
assert isinstance(x, E)

@Arrow(E.domain, F.domain)
def f(x):
    return ([2*x[0], (x[1] + x[0]) % 6] if x.dim == 1 
            else torch.stack(f(x.T)).T)

#-- pullback
fT = Tens.cofmap(f)
print(fT)
print(fT @ F.range()) 

#--- dual map: pushforward
Tf = fT.t()
print(Tf)
print(Tf @ E.ones())

TypeError: Input of type (Torus 3x3) not castable to Torus 3x3